In [1]:
import pandas as pd
import numpy as np
import shutil, os, re, sqlite3

<h2 style='color:#00A'>O código adiante obtém os arquivos csv, por tipo (rat, rat_evetivo, rat_viatura e rat_produtividade) e os ordena por data</h2>

In [2]:
# dir = os.getcwd()
# rats_files_names = os.listdir(dir+'/files/RAT')
# rat_pattern = re.compile('REDS_RAT_2')
# rat_viatura_pattern = re.compile('RAT_VIATURA')
# rat_efetivo_pattern = re.compile('RAT_EFETIVO')
# rat_produtividade_pattern = re.compile('RAT_Produtividade')

# rats_files = list()
# rats_viatura_files = list()
# rats_efetivo_files = list()
# rats_produtividade_files = list()

# for file_name in rats_files_names:
#     is_rat = rat_pattern.search(file_name) != None
#     is_rat_viatura = rat_viatura_pattern.search(file_name) != None
#     is_rat_efetivo = rat_efetivo_pattern.search(file_name) != None
#     is_rat_produtividade = rat_produtividade_pattern.search(file_name) != None
#     if is_rat:
#         rats_files.append(file_name)
#     elif is_rat_viatura:
#         rats_viatura_files.append(file_name)
#     elif is_rat_efetivo:
#         rats_efetivo_files.append(file_name)
#     elif is_rat_produtividade:
#         rats_produtividade_files.append(file_name)
        
# rats_files.sort(key=lambda name: int(name[9:17]))
# rats_viatura_files.sort(key=lambda name: int(name[18:26]))
# rats_efetivo_files.sort(key=lambda name: int(name[18:26]))
# rats_produtividade_files.sort(key=lambda name: int(name[23:31]))

<h2 style="color:#0A0;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT</h2>
<h3 style="color:#00A; margin:10px">Importa somente os dados do 23º BPM</h3>

<ol style="color:#222">
    <li>Lê o primeiro arquivo da lista e o passa para um DataFrame (df1)</li>
    <li>Lê o segundo arquivo da lista e o passa para outro DataFrame (df2)</li>
    <li>Concatena os dois DataFrames em um terceiro (df_acum)</li>
    <li>Itera sobre os demais arquivos, lendo cada um deles, aplicando strip(), a função de filtro do 23º BPM e depois o adiciona ao DataFrame df_acum</li>
    <li>Retira as duplicatas, pelo campo 'RAT.NUM_ATIVIDADE'</li>
    <li>Adiciona colunas data/hora início e término, tempo em datetime e tempo em inteiro </li>
    <li>Aplica .upper() nas colunas usadas para classificação.</li>
    <li>Classifica por Setores</li>
    <li>Classifica por Cias</li>
    <li>Insere os dados de df_acum no banco de dados gdo.db: (tabela tbl_rat)</li>

</ol>

In [18]:
df_classif['SETOR'] = df_classif['SETOR'].str.replace('SANTANENSE MORADA NOVA', 'SANTANENSE/MORADA NOVA')

In [23]:
# df_classif = pd.read_csv('files/classificadores.csv')
# df_classif = df_classif.apply(lambda col:col.str.upper())
# df_classif.set_index( df_classif['MUNICIPIO'] + " " + df_classif['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR'], inplace=True)
# df_classif['SETOR'] = df_classif['SETOR'].str.replace('LOURDES ITATIAIUCU', 'LOURDES/ITATIAIUCU')
# df_classif['SETOR'] = df_classif['SETOR'].str.replace('CENTRO PADRE EUSTAQUIO', 'CENTRO/PADRE EUSTAQUIO')
# df_classif['SETOR'] = df_classif['SETOR'].str.replace('SANTANENSE MORADA NOVA', 'SANTANENSE/MORADA NOVA')
# df_classif = df_classif.reset_index().drop_duplicates('index').set_index('index')


# def filter_23(df):
#     return df[
#         df['MUNICIPIO'].isin([
#             'DIVINOPOLIS',
#             'ITAUNA',
#             'ITATIAIUCU',
#             'CARMO DO CAJURU',
#             'SAO GONCALO DO PARA',
#             'CLAUDIO'
#         ])
#     ]


# df1 = pd.read_csv('files/RAT/' + rats_files[0], error_bad_lines=False, sep='|')
# df1 = df1.applymap(lambda x: x.strip() if type(x) == str else x)
# df1 = filter_23(df1)

# df2 = pd.read_csv('files/RAT/' + rats_files[1], error_bad_lines=False, sep='|')
# df2 = df2.applymap(lambda x: x.strip() if type(x) == str else x)
# df2 = filter_23(df2)

# df_acum = pd.concat([df1, df2])

# for rat_file in rats_files[2:]:
#     df_aux = pd.read_csv('files/RAT/' + rat_file, error_bad_lines=False, sep='|')
#     df_aux = df_aux.applymap(lambda x: x.strip() if type(x) == str else x)
#     df_aux = filter_23(df_aux)
#     df_acum = pd.concat([df_acum, df_aux])

# df_acum.drop_duplicates(subset='RAT.NUM_ATIVIDADE', keep='last', inplace=True)

# df_acum.loc[:,'DTA_HRA_INICIO'] = df_acum['DTA_INICIO'] + " " + df_acum['HRA_INICIO']
# df_acum.loc[:,'DTA_HRA_INICIO_DT'] = pd.to_datetime( df_acum['DTA_HRA_INICIO'], format='%d/%m/%Y %H:%M', errors='coerce')

# df_acum.loc[:,'DTA_HRA_TERMINO'] = df_acum.loc[:,'DTA_TERMINO'] + " " + df_acum.loc[:,'HRA_TERMINO']
# df_acum.loc[:,'DTA_HRA_TERMINO_DT'] = pd.to_datetime( df_acum.loc[:,'DTA_HRA_TERMINO'], format='%d/%m/%Y %H:%M', errors='coerce')

# df_acum.loc[:,'MES'] = df_acum.loc[:,'DTA_HRA_INICIO_DT'].dt.month
# df_acum.loc[:,'ANO'] = df_acum.loc[:,'DTA_HRA_INICIO_DT'].dt.year

# df_acum.loc[:,'TEMPO_DT'] = df_acum['DTA_HRA_TERMINO_DT'] - df_acum['DTA_HRA_INICIO_DT']

# df_acum.loc[:,'TEMPO_INT'] = df_acum['TEMPO_DT'].dt.total_seconds() / 60

# cols_classif = [
#     'MUNICIPIO',
#     'LOGRADOURO',
#     'DES_ENDERECO',
#     'COMPLEMENTO_ENDERECO',
#     'NOME_BAIRRO',
#     'LOGRADOURO2',
#     'DES_ENDERECO2'
# ]
# df_acum[cols_classif] = df_acum[cols_classif].apply(lambda col: col.str.upper())


# def classifica_setor(row):
#     mun = row['MUNICIPIO']
#     if mun == 'CLAUDIO':        
#         return 'CLAUDIO'
#     elif mun == 'ITATIAIUCU':
#         return 'LOURDES/ITATIAIUCU'
#     elif mun in ('CARMO DO CAJURU', 'SAO GONCALO DO PARA'):             
#         return 'CARMO DO CAJURU/SAO GONCALO DO PARA'    
#     elif ( mun + " N_RAT " + row['RAT.NUM_ATIVIDADE'] ) in df_classif.index:
#         return df_classif.loc[mun+" N_RAT "+row['RAT.NUM_ATIVIDADE'], 'SETOR']
#     elif mun + ' BAIRRO ' + row['NOME_BAIRRO'] in df_classif.index:       
#         return ( df_classif.loc[mun + ' BAIRRO ' + row['NOME_BAIRRO'].upper(), 'SETOR'] ).upper()
#     elif mun + ' LOGRADOURO ' + row['LOGRADOURO'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO ' + row['LOGRADOURO'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO ' + row['DES_ENDERECO'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO ' + row['DES_ENDERECO'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO2'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO2'].upper(), 'SETOR']
#     elif mun + ' COMPLEMENTO_END ' + row['COMPLEMENTO_ENDERECO'] in df_classif.index:
#         return df_classif.loc[mun + ' COMPLEMENTO_END ' + row['COMPLEMENTO_ENDERECO'].upper(), 'SETOR']
#     elif ( mun + ' COMPLEMENTO_END ' + row['DES_ENDERECO'] ) in df_classif.index:
#         return df_classif.loc[mun + ' COMPLEMENTO_END ' + row['DES_ENDERECO'], 'SETOR']    
#     else:
#         return 'other'

# df_acum['SETOR'] = df_acum.apply(lambda row: classifica_setor(row), axis=1)

# conds=[
#     df_acum['SETOR'].isin(['CENTRO/PADRE EUSTAQUIO','SANTANENSE/MORADA NOVA','LOURDES/ITATIAIUCU']),
#     df_acum['SETOR'].isin(['CENTRO','BOM PASTOR','ALTO GOIAS']),
#     df_acum['SETOR'].isin(['PLANALTO','SAO JOSE','CLAUDIO']),
#     df_acum['SETOR'].isin(['NITEROI','PORTO VELHO','CARMO DO CAJURU/SAO GONCALO DO PARA']),
    
# ]
# res=['51 CIA','53 CIA','139 CIA','142 CIA']
# df_acum['CIA'] = np.select(conds,res,default='other')

# df_acum.to_sql('tbl_rat', 'sqlite:///gdo.db', if_exists='replace')

/home/seo23bpm/.asdf/installs/python/anaconda3-2019.10/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  method=method,


<h2>Naturezas RQV e Lei seca</h2>

In [24]:
df_acum[    
    (df_acum['NAT.CODIGO'].str.contains('Y0700[1345]')) |
    (df_acum['NAT.CODIGO'] == 'Y04012')    
]['NAT.CODIGO'].value_counts()

Y07001    18783
Y07003     1141
Y04012      535
Y07004      126
Y07005       21
Name: NAT.CODIGO, dtype: int64

<h2 style="color:blue;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT_VEÍCULOS</h2>
<h3 style="color:red; margin:10px">Importa somente os dados do 23º BPM</h3>

### 1) Fazer descrição

In [14]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# df_acum = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_acum.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfv1 = pd.read_csv('files/RAT/' + rats_viatura_files[0], error_bad_lines=False, sep='|')
# dfv1 = dfv1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv1 = dfv1[dfv1['NUM_ATIVIDADE'].isin(df_rat['RAT.NUM_ATIVIDADE'])]

# dfv2 = pd.read_csv('files/RAT/' + rats_viatura_files[1], error_bad_lines=False, sep='|')
# dfv2 = dfv2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv2 = dfv2[dfv2['NUM_ATIVIDADE'].isin(df_rat['RAT.NUM_ATIVIDADE'])]

# dfv_acum = pd.concat([dfv1, dfv2])


# for rat_viatura_file in rats_viatura_files[2:]:
#     dfv_aux = pd.read_csv('files/RAT/'+rat_viatura_file, sep='|')
#     dfv_aux = dfv_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfv_aux = dfv_aux[dfv_aux['NUM_ATIVIDADE'].isin(df_rat['RAT.NUM_ATIVIDADE'])]
#     dfv_acum = pd.concat([dfv_acum, dfv_aux])
    
# dfv_acum.to_sql('tbl_rat_veiculo', 'sqlite:///gdo.db', if_exists='replace')

# df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
# df_ratv_unique = df_ratv['NUM_ATIVIDADE'].value_counts()
# df_ratv_unique.rename('VEICULOS', inplace=True)
# df_rat_e_v = df_rat.join(df_ratv_unique, how='left')
# df_rat_e_v['VEICULOS'].fillna(0, inplace=True)

# df_rat_e_v.groupby('NOM_UNIDADE_AREA').count()